In [1]:
! pip install scikit-learn-intelex


In [2]:
from sklearnex import patch_sklearn
# import logging
# logging.disable(logging.INFO)
patch_sklearn()

import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
PATH="/kaggle/input/webis-clickbait-22/"
sep='[SEP]'

label_encoder = LabelEncoder()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:

def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y = label_encoder.fit_transform(Y)
    else:
        Y = label_encoder.transform(Y)

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        i['text']=i['postText']+sep+i['targetParagraphs']+sep+i['targetTitle']+sep+i['targetDescription']+sep+i['targetKeywords']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y


In [4]:

x_train,y_train = get_split_data(PATH+'train.jsonl')
x_test,y_test = get_split_data(PATH+'test.jsonl',False)
x_val,y_val = get_split_data(PATH+'validation.jsonl',False)


len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [5]:
# len(x_train),len(y_train),len(x_test),len(y_test),len(x_val),len(y_val),len(x_train[0]),len(y_train[0])
x_train.shape,y_train.shape,x_test.shape,y_test.shape,x_val.shape,y_val.shape,len(x_train[10])


((3200,), (3200,), (1000,), (1000,), (800,), (800,), 3518)

In [6]:
type(x_train),type(y_train),type(x_test),type(y_test),type(x_val),type(y_val),type(x_train[0]),type(y_train[0])

(numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.str_,
 numpy.int64)

In [7]:
model2,encoder,vectorizer = pickle.load(open("/kaggle/input/random-forest/model_rf.pkl","rb"))
x_train =  vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)
x_val = vectorizer.transform(x_val)

INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [8]:
y_train_pred = model2.predict(x_train)
y_test_pred = model2.predict(x_test)
y_val_pred = model2.predict(x_val)
print("Training data metrics : ")
print(accuracy_score(y_train_pred,y_train))

print("Testing data accuracy")
print(accuracy_score(y_test,y_test_pred))

print("Testing data accuracy")
print(accuracy_score(y_val,y_val_pred))

INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Training data metrics : 
0.78875
Testing data accuracy
0.508
Testing data accuracy
0.49125


In [9]:
print("Training data metrics:")
print(balanced_accuracy_score(y_train_pred, y_train))
print(f1_score(y_train_pred, y_train, average='weighted'))
print(classification_report(y_train_pred, y_train))

print("Testing data metrics:")
print(balanced_accuracy_score(y_test_pred, y_test))
print(f1_score(y_test_pred, y_test, average='weighted'))
print(classification_report(y_test_pred, y_test))

print("Validation data metrics:")
print(balanced_accuracy_score(y_val_pred, y_val))
print(f1_score(y_val_pred, y_val, average='weighted'))
print(classification_report(y_val_pred, y_val))

Training data metrics:
0.8766706989247312
0.8020990054888002
              precision    recall  f1-score   support

           0       0.39      1.00      0.56       216
           1       0.74      0.94      0.83      1000
           2       1.00      0.69      0.82      1984

    accuracy                           0.79      3200
   macro avg       0.71      0.88      0.73      3200
weighted avg       0.88      0.79      0.80      3200

Testing data metrics:
0.5930952218563755
0.5566517240123232
              precision    recall  f1-score   support

           0       0.06      0.77      0.11        13
           1       0.40      0.51      0.45       316
           2       0.80      0.50      0.62       671

    accuracy                           0.51      1000
   macro avg       0.42      0.59      0.39      1000
weighted avg       0.66      0.51      0.56      1000

Validation data metrics:
0.5246176752988325
0.5445442234488008
              precision    recall  f1-score   support
